# Tutorial 7: 3D Lithospheric Model

**Romain Beucher** romain.beucher (at) anu.edu.au

In [1]:
from underworld import UWGeodynamics as GEO
from underworld import visualisation as vis

loaded rc file /home/romain/Projects/Project_Surface_processes/UWGeodynamics/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
u = GEO.UnitRegistry

In [3]:
# Characteristic values of the system
half_rate = 1.8 * u.centimeter / u.year
model_length = 400e3 * u.meter
surfaceTemp = 273.15 * u.degK
baseModelTemp = 1603.15 * u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2
KT = (baseModelTemp - surfaceTemp)

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT

In [4]:
Model = GEO.Model(elementRes=(32, 32, 16), 
                  minCoord=(0. * u.kilometer, 0. * u.kilometer, -170. * u.kilometer), 
                  maxCoord=(400. * u.kilometer, 400. * u.kilometer, 10. * u.kilometer), 
                  gravity=(0.0, 0.0, -9.81 * u.meter / u.second**2))

In [5]:
Model.outputDir="outputs_tutorial7"

In [6]:
Model.diffusivity = 1e-6 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

In [7]:
air        = Model.add_material(name="Air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=2 * u.kilometer))
stickyAir  = Model.add_material(name="StickyAir", shape=GEO.shapes.Layer3D(top=air.bottom, bottom= 0 * u.kilometer))
uppercrust = Model.add_material(name="UppperCrust", shape=GEO.shapes.Layer3D(top=stickyAir.bottom, bottom=-35 * u.kilometer))
mantleLithosphere = Model.add_material(name="MantleLithosphere", shape=GEO.shapes.Layer(top=uppercrust.bottom, bottom=-100 * u.kilometer))
mantle     = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=mantleLithosphere.bottom, bottom=Model.bottom))

In [8]:
if GEO.nProcs == 1:
    Fig = vis.Figure(resolution=(1200,600))
    Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=0.5)
    Fig.window()

Error handling request
Traceback (most recent call last):
  File "/home/romain/Projects/Project_Surface_processes/surface_processes/lib/python3.9/site-packages/aiohttp/web_protocol.py", line 314, in data_received
    messages, upgraded, tail = self._request_parser.feed_data(data)
  File "aiohttp/_http_parser.pyx", line 546, in aiohttp._http_parser.HttpParser.feed_data
aiohttp.http_exceptions.BadStatusLine: 400, message="Bad status line 'invalid HTTP method'"
Error handling request
Traceback (most recent call last):
  File "/home/romain/Projects/Project_Surface_processes/surface_processes/lib/python3.9/site-packages/aiohttp/web_protocol.py", line 314, in data_received
    messages, upgraded, tail = self._request_parser.feed_data(data)
  File "aiohttp/_http_parser.pyx", line 546, in aiohttp._http_parser.HttpParser.feed_data
aiohttp.http_exceptions.BadStatusLine: 400, message="Bad status line 'invalid HTTP method'"


In [9]:
air.diffusivity = 1.0e-6 * u.metre**2 / u.second
stickyAir.diffusivity = 1.0e-6 * u.metre**2 / u.second

air.capacity = 100. * u.joule / (u.kelvin * u.kilogram)
stickyAir.capacity = 100. * u.joule / (u.kelvin * u.kilogram)

In [10]:
air.density         = 1. * u.kilogram / u.metre**3
stickyAir.density   = 1. * u.kilogram / u.metre**3
uppercrust.density  = 2620. * u.kilogram / u.metre**3
mantleLithosphere.density  = 3370. * u.kilogram / u.metre**3
mantle.density      = 2620. * u.kilogram / u.metre**3

In [11]:
uppercrust.radiogenicHeatProd = 0.7 * u.microwatt / u.meter**3
mantleLithosphere.radiogenicHeatProd = 0.02e-6 * u.microwatt / u.meter**3

In [12]:
rh = GEO.ViscousCreepRegistry()

In [13]:
air.viscosity                = 1e19 * u.pascal * u.second
stickyAir.viscosity          = 1e20 * u.pascal * u.second
uppercrust.viscosity         = 1e23 * u.pascal * u.second
mantleLithosphere.viscosity  = 1e20 * u.pascal * u.second
mantle.viscosity             = 1e19 * u.pascal * u.second

## Temperature Boundary Condition

In [14]:
Model.set_temperatureBCs(top=293.15 * u.degK, 
                         bottom=1603.15 * u.degK)

## Velocity Boundary Conditions

In [15]:
Model.set_velocityBCs(left=[-2.5 * u.centimeter / u.year, None, None],
                       right=[2.5 * u.centimeter / u.year, None, None],
                       front=[None, 0.0, None], back=[None, 0.0, None],
                       bottom=[None, None, 0.0],
                       order_wall_conditions=["bottom", "top", "front", "back", "left", "right"])

In [16]:
Model.init_model(temperature="steady-state", pressure="lithostatic")

In [17]:
Model.solver.set_inner_method("mumps")
Model.solver.set_penalty(1e6)
GEO.rcParams["initial.nonlinear.tolerance"] = 1e-4

In [ ]:
if GEO.nProcs == 1:
    Fig = vis.Figure(resolution=(1200,600))
    Fig.Surface(Model.mesh, Model.temperature, cullface=False, opacity=0.5, colours="coolwarm")
    Fig.window()

In [ ]:
if GEO.nProcs == 1:
    Fig = vis.Figure(resolution=(1200,600))
    Fig.Surface(Model.mesh.subMesh, Model.pressureField, cullface=False, opacity=0.5)
    Fig.window()

In [20]:
Model.run_for(nstep=1)

Running with UWGeodynamics version 2.10.0-dev-71e6210(2.10-pre_release)
Options:  -Q22_pc_type gkgdiag -force_correction True -ksp_type bsscr -pc_type none -ksp_k2_type NULL -rescale_equations False -remove_constant_pressure_null_space False -change_backsolve False -change_A11rhspresolve False -restore_K False -A11_ksp_type preonly -A11_pc_type lu -A11_pc_factor_mat_solver_type mumps -scr_ksp_type fgmres -scr_ksp_rtol 1e-05 -A11_mg_active False
Step:     1 Model Time: 120699.1 year dt: 120699.1 year (2020-08-13 05:59:23)


1

In [ ]:
if GEO.nProcs == 1:
    Fig = vis.Figure(resolution=(1200,600))
    Fig.VectorArrows(Model.mesh, Model.velocityField, cullface=False, opacity=0.5)
    Fig.window()